In [ ]:
import DataSetQuery
import MalardHelpers
import datetime

#Sets the URL of the Malard ServiceGateway.
query = DataSetQuery.DataSetQuery('http://localhost:9000','DEV')



In [ ]:
#Setup the bounding box
import pandas as pd

minX=0
maxX=100000
minY=0
maxY=100000
minT=datetime.datetime(2012,1,1,0,0)
maxT=datetime.datetime(2012,1,31,0,0)

#a list of columns to see in the output (x,y,time will be added)
projections = ['lat','lon','elev','power','coh','swathFileId'] 
#a list of dictionaries that specify the filters
#only numeric fields are supported with the following operations gt(greater than), gte ( greater than equals), lt (less than) and lte (less than equals)
#filters are treated as and conditions
filters = [{'column':'swathFileId','op':'gte','threshold':11783}]

#No filters
nofiltersTandemX = query.executeQuery( 'mtngla','tandemx',minX,maxX,minY,maxY,minT,maxT,projections,filters)
print(nofiltersTandemX)
dfTx = MalardHelpers.getDataFrameFromNetCDF(nofiltersTandemX)

nofiltersSrtm = query.executeQuery( 'mtngla','srtm',minX,maxX,minY,maxY,minT,maxT,projections,filters)
#Converts the NetCDF into a DataFrame.
dfSrtm = MalardHelpers.getDataFrameFromNetCDF(nofiltersSrtm)

print("TandemX MaxElev:[%f] MinElev:[%f] Points:[%d]" % (dfTx['elev'].max(),dfTx['elev'].min(),dfTx['elev'].count()))
print("Srtm MaxElev:[%f] MinElev:[%f] Points:[%d]" % (dfSrtm['elev'].max(),dfSrtm['elev'].min(),dfSrtm['elev'].count()))


df = pd.merge(dfTx,dfSrtm,left_on=['lat','lon'],right_on=['lat','lon'],how='left')

df.to_csv("/data/puma1/scratch/test/alljoin.csv")

query.releaseCache(nofiltersTandemX)
query.releaseCache(nofiltersSrtm)



In [ ]:
print(dfTx.groupby(['swathFileId']).agg(['count']))

In [ ]:
print(dfSrtm.groupby(['swathFileId']).agg(['count']))

In [ ]:

sourceTandemX = '/data/puma1/scratch/test/tandemx/CS_LTA__SIR_SIN_2S_20120122T040316_20120122T040631_C001.nc'
sourceSrtm = '/data/puma1/scratch/test/srtm/CS_LTA__SIR_SIN_2S_20120122T040316_20120122T040631_C001.nc'

rawTx = MalardHelpers.getDataFrameFromNetCDF(sourceTandemX)
rawSrtm = MalardHelpers.getDataFrameFromNetCDF(sourceSrtm)


In [ ]:
print(rawTx['lat'].count())
print(rawSrtm['lat'].count())

print(rawTx['lon'].sum())
print(rawSrtm['lon'].sum())



In [ ]:
df = pd.merge(rawTx,rawSrtm,left_on=['lat','lon'],right_on=['lat','lon'],how='inner')

In [ ]:
df.info()

In [ ]:
df['lat'].count()